In [ ]:
import datetime
import numpy
import pygetm

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

In [ ]:
# Set up rectangular domain with outer points masked
domain = pygetm.domain.create_cartesian(500.*numpy.arange(101), 500.*numpy.arange(31), 49, lat=0, H=50)
sim = pygetm.Simulation(domain, runtype=pygetm.BAROTROPIC_3D, advection_scheme=1)

# Idealized surface forcing
tausx = domain.U.array(fill=0.01)
tausy = domain.V.array(fill=0.)
sp = domain.T.array(fill=0.)

idpdx = domain.U.array(fill=0., z=pygetm.CENTERS)
idpdy = domain.V.array(fill=0., z=pygetm.CENTERS)
viscosity = domain.T.array(fill=1e-4, z=pygetm.INTERFACES)

# Time
timestep = 10.
ntime = int(3600. // timestep)
mode_split = 5
domain.T.zio.all_values[...] = 0
domain.T.zin.all_values[...] = 0
sim.start(datetime.datetime(2000, 1, 1), timestep, mode_split)

# Set up velocity plot
nsample_x = 2
nsample_z = 2
iy = 0
fig, ax = matplotlib.pyplot.subplots()
u = sim.pk.interp(domain.T) / domain.T.H
w = sim.ww.interp(domain.T, z=pygetm.CENTERS)
z_2d = domain.T.zc[::nsample_z, iy, ::nsample_x]
x_2d = numpy.broadcast_to(domain.T.x[iy, ::nsample_x], z_2d.shape)
Q = ax.quiver(x_2d, z_2d, u[::nsample_z, iy, ::nsample_x], w[::nsample_z, iy, ::nsample_x], scale=.001)
title = ax.set_title('time: 0 s')

In [ ]:
times = timestep * numpy.arange(ntime)
plotting_interval = 5
for istep, time in enumerate(times):
    sim.update_surface_pressure_gradient(domain.T.z, sp)
    sim.uv_momentum_2d(timestep, tausx, tausy, sim.dpdx, sim.dpdy)
    sim.update_sealevel(timestep, sim.U, sim.V)
    sim.update_depth()

    if istep % mode_split == 0:
        sim.Ui.all_values[...] /= mode_split
        sim.Vi.all_values[...] /= mode_split
        sim.start_3d()
        domain.do_vertical()
        sim.update_surface_pressure_gradient(domain.T.zio, sp)
        sim.uvw_momentum_3d(timestep * mode_split, tausx, tausy, sim.dpdx, sim.dpdy, idpdx, idpdy, viscosity)
        sim.Ui.all_values[...] = 0
        sim.Vi.all_values[...] = 0

    # Update velocity plot
    if istep % plotting_interval == 0:
        u = sim.pk.interp(domain.T) / domain.T.H
        sim.ww.interp(w)
        Q.set_UVC(u[::nsample_z, iy, ::nsample_x], w[::nsample_z, iy, ::nsample_x])
        title.set_text('time: %s s' % time)
        fig.canvas.draw()